# Extrapolação XGBoost c/ predict_proba()

## Projeto de Mapeamento Marinho Apresentado Nas Jornadas de Engenharia Hidrográfica

## Diogo Ceddia Porto Silva

#### Importação de bibliotecas

In [1]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import time
from sklearn import metrics
import numpy as np
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")

#### Importação dos dados

In [2]:
dado_linhas = pd.read_csv("C:\DCPS\GitHub\Dados_MapeamentoIA_JEH\dado_linhas.txt")
dado_tudo = pd.read_csv("C:\DCPS\GitHub\Dados_MapeamentoIA_JEH\dado_tudo.txt")

#### Separando target/feature e treino/validação

In [3]:
features = ['z','bs','sl','as','dist_lc','dist_ds']
target = ['classe']

X_test = dado_tudo[features]
X      = dado_linhas[features]
y      = dado_linhas[target]

X = StandardScaler().fit_transform(X)
X = pd.DataFrame(X,columns=features)
X_test = StandardScaler().fit_transform(X_test)
X_test = pd.DataFrame(X_test,columns=features)

#### Definição do modelo 

In [4]:
params = {'learning_rate': 0.62,
          'max_depth': 7,
          'n_estimators': 1310
         }

model = XGBClassifier(use_label_encoder=False,**params)

#### Treinamento e predição

In [5]:
start = time.time()
model.fit(X, y.values.ravel(), eval_metric='merror')
y_pred_all = model.predict(X_test)
end=time.time()
print('Tempo:',end-start,'segundos')

Tempo: 263.44949293136597 segundos


In [6]:
y_pred_all = pd.DataFrame(y_pred_all,columns=['classe'])
y_pred_all['proba_rocha'] = model.predict_proba(X_test)[:,1]

#### Filtro estatístico

In [7]:
for i in [0.90, 0.95, 0.99, 0.999]:

    pred = y_pred_all.copy()

    pred.classe.iloc[np.where(pred['proba_rocha']>i)[0]] = 1
    pred.classe.iloc[np.where(pred['proba_rocha']<i)[0]] = 0

    print(' ')
    print(len(np.where(pred.classe==1)[0]))
    print(len(np.where(pred.classe==0)[0]))
    
    proba_rocha = pd.concat([dado_tudo[['utmx','utmy']],pred[['classe']]],axis=1)
    proba_rocha.to_csv(rf"C:\DCPS\GitHub\Dados_MapeamentoIA_JEH\proba_rocha_filtrado{i}.txt",index=False,header=False)

 
2738025
3181095
 
2633607
3285513
 
2392043
3527077
 
2028095
3891025


#### Salvando predição

In [8]:
pred_xy0_all = dado_tudo.iloc[np.where(y_pred_all.classe == 0)[0]].reset_index(drop=True)
pred_xy0_all['classe']= 0
pred_xy1_all = dado_tudo.iloc[np.where(y_pred_all.classe == 1)[0]].reset_index(drop=True)
pred_xy1_all['classe']= 1
print(len(pred_xy0_all))
print(len(pred_xy1_all))

2892958
3026162


In [9]:
#pred_xy0_all.to_csv(r"C:\DCPS\GitHub\Dados_MapeamentoIA_JEH\pred_sedimento_v2_tuned.txt",
#                    index=False,header=False,columns=['utmx','utmy'])
#pred_xy1_all.to_csv(r"C:\DCPS\GitHub\Dados_MapeamentoIA_JEH\pred_rocha_v2_tuned.txt",
#                    index=False,header=False,columns=['utmx','utmy'])

In [10]:
#proba_rocha = pd.concat([dado_tudo[['utmx','utmy']],y_pred_all[['proba_rocha']]],axis=1)
#proba_rocha.to_csv(r"C:\DCPS\GitHub\Dados_MapeamentoIA_JEH\proba_rocha.txt",index=False,header=False)